In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Thư viện :

In [4]:
!pip install torch onnx safetensors diffusers transformers
# !pip install torch onnx safetensors

In [5]:
from huggingface_hub import snapshot_download

# Đường dẫn thư mục lưu trên Google Drive
save_path = "vae"

# Tải toàn bộ thư mục `unet`
snapshot_download(
    repo_id="timbrooks/instruct-pix2pix",
    local_dir=save_path,
    allow_patterns=["vae/*"]  # Chỉ tải thư mục `unet`
)

print(f"✅ Đã tải xong thư mục `vae` vào {save_path}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae%2Fconfig.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

✅ Đã tải xong thư mục `vae` vào vae


In [6]:
!pip install  accelerate

In [ ]:
import torch
import os
import json
from diffusers import AutoencoderKL
from safetensors.torch import load_file

#  Đường dẫn tới model VAE và config
safetensor_path = "/content/vae/vae/diffusion_pytorch_model.fp16.safetensors"
config_path = "/content/vae/vae/config.json"

#  Nơi lưu file ONNX
onnx_dir = "/content/drive/MyDrive/Model/vae/model_onnx"
os.makedirs(onnx_dir, exist_ok=True)
onnx_path = os.path.join(onnx_dir, "vae.onnx")

#  Load config từ file JSON
with open(config_path, "r") as f:
    config = json.load(f)

#  Tạo mô hình VAE từ config
vae = AutoencoderKL(**config)

#  Load trọng số từ file safetensors
state_dict = load_file(safetensor_path)

#  In danh sách key của state_dict 
print(" Danh sách key trong state_dict:")
print(list(state_dict.keys())[:20])  

# Load lại state_dict với strict=False 
vae.load_state_dict(state_dict, strict=False)

#  Chuyển mô hình sang chế độ đánh giá
vae.eval()

# Tạo tensor input  từ config
batch_size = 1
in_channels = config["in_channels"]
height = config["sample_size"]
width = config["sample_size"]

dummy_input = torch.randn(batch_size, in_channels, height, width, dtype=torch.float32)

#  Tạo Wrapper để đưa vào ONNX
class VAEWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model.encode(x).latent_dist.sample()

wrapped_vae = VAEWrapper(vae)

# Xuất mô hình VAE sang ONNX
torch.onnx.export(
    wrapped_vae,
    (dummy_input,),
    onnx_path,
    export_params=True,
    opset_version=16,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["latent"],
    dynamic_axes={"input": {0: "batch_size"}, "latent": {0: "batch_size"}}
)

print(f"✅ Convert thành công! Mô hình ONNX VAE đã lưu tại: {onnx_path}")


 Danh sách key trong state_dict:
['decoder.conv_in.bias', 'decoder.conv_in.weight', 'decoder.conv_norm_out.bias', 'decoder.conv_norm_out.weight', 'decoder.conv_out.bias', 'decoder.conv_out.weight', 'decoder.mid_block.attentions.0.group_norm.bias', 'decoder.mid_block.attentions.0.group_norm.weight', 'decoder.mid_block.attentions.0.to_k.bias', 'decoder.mid_block.attentions.0.to_k.weight', 'decoder.mid_block.attentions.0.to_out.0.bias', 'decoder.mid_block.attentions.0.to_out.0.weight', 'decoder.mid_block.attentions.0.to_q.bias', 'decoder.mid_block.attentions.0.to_q.weight', 'decoder.mid_block.attentions.0.to_v.bias', 'decoder.mid_block.attentions.0.to_v.weight', 'decoder.mid_block.resnets.0.conv1.bias', 'decoder.mid_block.resnets.0.conv1.weight', 'decoder.mid_block.resnets.0.conv2.bias', 'decoder.mid_block.resnets.0.conv2.weight']


/usr/local/lib/python3.11/dist-packages/diffusers/models/downsampling.py:136: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/usr/local/lib/python3.11/dist-packages/diffusers/models/downsampling.py:145: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/usr/local/lib/python3.11/dist-packages/torch/onnx/_internal/jit_utils.py:308: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Trigge

✅ Convert thành công! Mô hình ONNX VAE đã lưu tại: /content/drive/MyDrive/Model/vae/model_onnx/vae.onnx


In [8]:
import os

onnx_path = "/content/drive/MyDrive/Model/vae/model_onnx/vae.onnx"
print("ONNX File Exists:", os.path.exists(onnx_path))


ONNX File Exists: True


In [9]:
!pip install onnxruntime

In [10]:
import onnxruntime as ort

onnx_path = "/content/drive/MyDrive/Model/vae/model_onnx/vae.onnx"

# Load ONNX bằng onnxruntime (ít tốn RAM hơn)
session = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])
print("🚀 ONNX model đã load thành công!")


🚀 ONNX model đã load thành công!
